In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# 상품 정보를 수집하는 사용자 정의 함수
def scrape_products(driver, target_count):
    # 빈 리스트와 중복 확인을 위한 set
    box = []
    collected_items = set()

    # 상품 수집 반복
    while len(box) < target_count:
        # 페이지를 아래로 스크롤
        body = driver.find_element(By.CSS_SELECTOR, 'body')
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)  # 페이지 로드 대기

        # BeautifulSoup으로 HTML 파싱
        html = BeautifulSoup(driver.page_source, 'html.parser')

        # 상품 정보 추출
        products = html.find_all('div', class_='_2a0vHVAYuk')  # 해당 클래스명을 가진 상품 영역

        for product in products:
            title_tag = product.find('strong', class_='_1lxNNH5gfD') # 제목
            price_tag = product.find('span', class_='_3MzrqSUlwR')   # 가격
            rating_tag = product.find('span', class_='_29gNnX1yLx')  # 평점

            if title_tag and price_tag and rating_tag:
                # 상품 제목, 가격, 평점 추출
                title = title_tag.get_text(strip=True)
                price = price_tag.get_text(strip=True).replace('원', '')
                rating = rating_tag.get_text(strip=True).replace('평점', '')

                # 중복된 상품인지 확인 후 저장
                if title not in collected_items:
                    box.append({
                        'title': title,
                        'price': price,
                        'rating': rating
                    })
                    collected_items.add(title)

            # 100개 이상 수집되면 중단
            if len(box) >= target_count:
                break

    # DataFrame으로 변환하여 반환
    return pd.DataFrame(box)




In [6]:
# WebDriver 설정
import undetected_chromedriver as uc
driver = uc.Chrome()

# 사이트로 이동
driver.get("https://shopping.naver.com/fresh/directfarm/home")  # 크롤링할 사이트의 URL 입력

# 상품 정보 수집 함수 호출
df = scrape_products(driver, target_count=101) # 상품 101개

# 데이터 프레임을 Excel 파일로 저장
df.to_excel('scraped_products(naver).xlsx', index=False)

df

,title,price,rating
0,국산 흰다리새우 1kg 생물 왕새우 활새우 대하철,"22,900",4.91
1,국산 흰다리새우 1kg 왕새우 제철,"21,900",4.84
2,충북 보은 대추칩 80g [10봉구매시+1봉추가],"4,990",4.84
3,문경 생 오미자 생과 2024년 세척 10kg,"149,900",4.79
4,꽃게 가을 알있는 활암꽃게 냉동 절단 몽당 1kg 경인서부수협 10번,"14,900",4.77
...,...,...,...
96,[신선특허] 내일받는 기삼전복 1kg 8미부터 60미 손질도구 포함,"39,900",4.83
97,거제도 해녀 자연산 바위굴 생굴 벚굴 하프셀 굴 석화,"36,900",4.62
98,서산 생강 2kg 흙생강 24년 햇생강,"21,900",4.7
99,영양가득 둥근마 3kg / 5kg 마 참마 장마,"45,900",4.62


title
국산 흰다리새우 1kg 생물 왕새우 활새우 대하철                         1
친환경 유기농 알밤 2kg 대사이즈 믿고 먹는 햇밤                        1
자연산 왕대하 새우 1kg 20마리~25마리                            1
동해안 총알 통찜 오징어 생물 냉동 1kg 국산                          1
유기농 국산 햇 땅콩 500g / 삶은땅콩 급냉 냉동 바로먹는 땅콩 구룡농원          1
                                                   ..
[재벌사과] 믿고 껍질째먹는 청송 얼음골 명품 홍로 꿀 사과 5kg               1
[24년 햅쌀] 유기농 누룽지쌀 백세미 10kg 6kg 4kg 골드퀸 수향미 골든퀸3호    1
나주배 가정용 5kg 3kg 신고배 2024년산 햇배                       1
꼼꼼하게 선별한 충남서산 생강 1kg 특품                             1
국내산 제철 흰다리새우 1kg 왕새우 새우철                            1
Name: count, Length: 101, dtype: int64